## Q1. Running Elastic Search

What is the version.build_hash value of the elastic_search?

Answer: 42f05b9372a9a4a470db3b52817899b99a76ee73

### Getting the data

In [1]:
import requests 

docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

### Q2. Indexing the data

In [8]:
from elasticsearch import Elasticsearch
from tqdm.auto import tqdm

In [7]:
es_client = Elasticsearch(hosts="http://localhost:9200")

index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },

    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name = "course_questions"

es_client.indices.create(
    index=index_name,
    body=index_settings
)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course_questions'})

In [9]:
for doc in tqdm(documents):
    es_client.index(
        index=index_name,
        document=doc
    )

100%|██████████| 948/948 [00:06<00:00, 144.94it/s]


### Q3. Searching the data

In [12]:
query = "How do I execute a command in a running docker container?"

search_query = {
    "size": 5,
    "query": {
        "bool": {
            "must": {
                "multi_match": {
                    "query": query,
                    "fields": ["question^4", "text"],
                    "type": "best_fields"
                }
            }
        }
    }
}

In [14]:
response = es_client.search(
    index=index_name,
    body=search_query
)

print(response)

{'took': 30, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 865, 'relation': 'eq'}, 'max_score': 84.050095, 'hits': [{'_index': 'course_questions', '_id': 'ArMhcJABG6tqRuBbV-Bg', '_score': 84.050095, '_source': {'text': 'Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)', 'section': '5. Deploying Machine Learning Models', 'question': 'How do I debug a docker container?', 'course': 'machine-learning-zoomcamp'}}, {'_index': 'course_questions', '_id': 'ELMhcJABG6tqRuBbSd7-', '_score': 75.54128, '_source': {'text': 'In case running pgcli  locally causes issues or you do not want to install it locally you can use it running in a Docker container instead.

In [18]:
print(f"The score of the top ranking result is: {response['hits']['max_score']}")

The score of the top ranking result is: 84.050095


### Q4. Filtering

In [27]:
query = "How do I execute a command in a running docker container?"

search_query = {
    "size": 3,
    "query": {
        "bool": {
            "must": {
                "multi_match": {
                    "query": query,
                    "fields": ["question^4", "text"],
                    "type": "best_fields"
                }
            },
            "filter": {
                "term": {
                    "course": "machine-learning-zoomcamp"
                }
            }
        }
    }
}

response = es_client.search(
    index=index_name,
    body=search_query
)

third_response = response['hits']['hits'][-1]['_source']['question'].strip()

print(f"The third document in the response is \n{third_response}")

The third document in the response is 
How do I copy files from a different folder into docker container’s working directory?


### Q5. Building a prompt

In [31]:
context_template = """
Q: {question}
A: {text}
""".strip()

prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT:
{context}
""".strip()

context = ""

for hit in response["hits"]["hits"]:
    context += context_template.format(question=hit["_source"]["question"], text=hit["_source"]["text"]) + "\n\n"

prompt = prompt_template.format(question=query, context=context).strip()

print(len(prompt))

1462


### Q6. Tokens

In [32]:
import tiktoken

In [33]:
encoding = tiktoken.encoding_for_model("gpt-4o")

Encoding into tokens

In [35]:
prompt_tokens = encoding.encode(prompt)

print(len(prompt_tokens))

322


Decoding tokens

In [37]:
encoding.decode_single_token_bytes(63842).decode(encoding="utf-8")

"You're"